In [1]:
!pip install gensim

In [2]:
!pip install tqdm

In [66]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

In [67]:
df = pd.read_csv(r'medicines_dataset.csv')
df.head(10)

,id,name,price(₹),Is_discontinued,manufacturer_name,type,pack_size_label,short_composition1,short_composition2
0,1,Augmentin 625 Duo Tablet,223.42,False,Glaxo SmithKline Pharmaceuticals Ltd,allopathy,strip of 10 tablets,Amoxycillin (500mg),Clavulanic Acid (125mg)
1,2,Azithral 500 Tablet,132.36,False,Alembic Pharmaceuticals Ltd,allopathy,strip of 5 tablets,Azithromycin (500mg),NaN
2,3,Ascoril LS Syrup,118.00,False,Glenmark Pharmaceuticals Ltd,allopathy,bottle of 100 ml Syrup,Ambroxol (30mg/5ml),Levosalbutamol (1mg/5ml)
3,4,Allegra 120mg Tablet,218.81,False,Sanofi India Ltd,allopathy,strip of 10 tablets,Fexofenadine (120mg),NaN
4,5,Avil 25 Tablet,10.96,False,Sanofi India Ltd,allopathy,strip of 15 tablets,Pheniramine (25mg),NaN
5,6,Allegra-M Tablet,241.48,False,Sanofi India Ltd,allopathy,strip of 10 tablets,Montelukast (10mg),Fexofenadine (120mg)
6,7,Amoxyclav 625 Tablet,223.27,False,Abbott,allopathy,strip of 10 tablets,Amoxycillin (500mg),Clavulanic Acid (125mg)
7,8,Azee 500 Tablet,132.38,False,Cipla Ltd,allopathy,strip of 5 tablets,Azithromycin (500mg),NaN
8,9,Atarax 25mg Tablet,85.50,False,Dr Reddy's Laboratories Ltd,allopathy,strip of 15 tablets,Hydroxyzine (25mg),NaN
9,10,Ascoril D Plus Syrup Sugar Free,129.00,False,Glenmark Pharmaceuticals Ltd,allopathy,bottle of 100 ml Syrup,Phenylephrine (5mg),Chlorpheniramine Maleate (2mg)


DESCRIPTIVE ANALYSIS

In [68]:
df.shape

(253973, 9)

In [69]:
# Unique values
unique_values= df.nunique()
unique_values

id                    253973
name                  249398
price(₹)               20664
Is_discontinued            2
manufacturer_name       7648
type                       1
pack_size_label         1929
short_composition1      8523
short_composition2      2980
dtype: int64

In [70]:
# Missing data
df.isnull().sum()

id                         0
name                       0
price(₹)                   0
Is_discontinued            0
manufacturer_name          0
type                       0
pack_size_label            0
short_composition1         0
short_composition2    141802
dtype: int64

In [71]:
df[df.Is_discontinued==True].shape

(7905, 9)

DATA CLEANING

In [72]:
# 'type' column has only 1 value and will not have an effect on the final model
df=df.drop(columns=['type'])
# 'Is_discontinued','id' can also be dropped
df = df[df.Is_discontinued==False]
df=df.drop(columns=['Is_discontinued','id'])

In [73]:
df.head()

,name,price(₹),manufacturer_name,pack_size_label,short_composition1,short_composition2
0,Augmentin 625 Duo Tablet,223.42,Glaxo SmithKline Pharmaceuticals Ltd,strip of 10 tablets,Amoxycillin (500mg),Clavulanic Acid (125mg)
1,Azithral 500 Tablet,132.36,Alembic Pharmaceuticals Ltd,strip of 5 tablets,Azithromycin (500mg),NaN
2,Ascoril LS Syrup,118.00,Glenmark Pharmaceuticals Ltd,bottle of 100 ml Syrup,Ambroxol (30mg/5ml),Levosalbutamol (1mg/5ml)
3,Allegra 120mg Tablet,218.81,Sanofi India Ltd,strip of 10 tablets,Fexofenadine (120mg),NaN
4,Avil 25 Tablet,10.96,Sanofi India Ltd,strip of 15 tablets,Pheniramine (25mg),NaN


In [74]:
# Exploring the 'pack_size_label' column
form=[]
count={}
for i in df.pack_size_label:
    words=i.split()
    x=words[-1].lower()
    if len(x)<=2:
        x=words[-2].lower()
    if x in form:
        count[x]+=1
    else:
        count[x]=1
        form.append(x)

sorted_count = dict(sorted(count.items(), key=lambda x:x[1], reverse=True))
print(sorted_count)

{'tablets': 129641, 'injection': 30931, 'tablet': 18270, 'syrup': 16409, 'capsules': 15236, 'suspension': 9011, 'capsule': 6202, 'cream': 4863, 'drop': 3697, 'solution': 1625, 'gel': 1580, 'ointment': 1574, 'drops': 1388, 'lotion': 989, 'infusion': 836, 'soap': 515, 'powder': 344, 'spray': 331, 'expectorant': 321, 'shampoo': 261, 'kit': 229, 'inhaler': 211, 'wash': 193, 'liquid': 140, 'injections': 126, 'granules': 115, 'respules': 111, 'suppositories': 70, 'rotacaps': 51, 'sachet': 51, 'gargle': 51, 'patches': 47, 'strips': 37, 'suppository': 32, 'paint': 32, 'inhalation': 28, 'syringe': 27, 'strip': 25, 'respicaps': 24, 'transcaps': 23, 'emulsion': 22, 'lozenges': 21, 'linctus': 19, 'pessaries': 18, 'transhaler': 18, 'paste': 17, 'combikit': 17, 'bottles': 16, 'toothpaste': 16, 'patch': 16, 'suspensions': 12, 'gums': 12, 'transpules': 11, 'combipack': 9, 'scrub': 9, 'solutions': 9, 'serum': 9, 'jelly': 8, 'lacquer': 7, 'tabcaps': 7, 'sachets': 7, 'enema': 7, 'syrups': 6, 'husk': 6, '

In [75]:
# Fix: removing the plural issue (eg: tablet | tablets)
final_count=sorted_count.copy()
for key,value in sorted_count.items():
    test=(key+'s') in final_count
    if test:
        final_count[key]+=final_count[key+'s']
        del final_count[key+'s']
        form.remove(key+'s')
        
final_count = dict(sorted(final_count.items(), key=lambda x:x[1], reverse=True))
print(final_count)

{'tablet': 147911, 'injection': 31057, 'capsule': 21438, 'syrup': 16415, 'suspension': 9023, 'drop': 5085, 'cream': 4867, 'solution': 1634, 'gel': 1581, 'ointment': 1574, 'lotion': 990, 'infusion': 837, 'soap': 515, 'powder': 344, 'spray': 331, 'expectorant': 321, 'shampoo': 262, 'kit': 229, 'inhaler': 211, 'wash': 193, 'liquid': 140, 'granules': 115, 'respules': 111, 'suppositories': 70, 'strip': 62, 'sachet': 58, 'rotacaps': 51, 'gargle': 51, 'patches': 47, 'suppository': 32, 'paint': 32, 'inhalation': 28, 'syringe': 27, 'respicaps': 24, 'transcaps': 23, 'emulsion': 23, 'lozenges': 21, 'linctus': 19, 'pessaries': 18, 'transhaler': 18, 'paste': 17, 'combikit': 17, 'bottle': 17, 'toothpaste': 16, 'patch': 16, 'gum': 14, 'transpules': 11, 'combipack': 9, 'scrub': 9, 'serum': 9, 'jelly': 8, 'lacquer': 7, 'tabcaps': 7, 'enema': 7, 'husk': 6, 'liniment': 6, 'cartriges': 6, 'caplets': 5, 'redicaps': 5, 'instacap': 5, 'pastilles': 4, 'foam': 4, 'nexcaps': 4, 'pellets': 4, 'pouch': 4, 'elixir

In [76]:
# Extract unique pack sizes from 'pack_size_label' column
pack_sizes = df['pack_size_label'].str.extract(r'(\b\w+\b)')[0].unique()

def getType(row):
    for pack_size in pack_sizes:
        if pack_size in row['pack_size_label'].lower():
            return pack_size

def getComp(row):
    return row['short_composition1'].split('(')[0].lower().rstrip()

def getValue(row):
    return row['short_composition1'].split('(')[1].lower().rstrip().rstrip(')')

df['type'] = df.apply(lambda row: getType(row), axis=1)
df['primary_comp'] = df.apply(lambda row: getComp(row), axis=1)
df['entire_comp'] = df['primary_comp'] + ' ' + df['short_composition2'].astype(str).apply(lambda x: x.split('(')[0].lower().rstrip() if pd.notna(x) else '')
df['value'] = df.apply(lambda row: getValue(row), axis=1)
df['entire_value'] = df['value'] + ' ' + df['short_composition2'].astype(str).apply(lambda x: x.split('(')[1].lower().rstrip().rstrip(')') if pd.notna(x) and len(x.split('(')) > 1 else '')

df.head()

,name,price(₹),manufacturer_name,pack_size_label,short_composition1,short_composition2,type,primary_comp,entire_comp,value,entire_value
0,Augmentin 625 Duo Tablet,223.42,Glaxo SmithKline Pharmaceuticals Ltd,strip of 10 tablets,Amoxycillin (500mg),Clavulanic Acid (125mg),strip,amoxycillin,amoxycillin clavulanic acid,500mg,500mg 125mg
1,Azithral 500 Tablet,132.36,Alembic Pharmaceuticals Ltd,strip of 5 tablets,Azithromycin (500mg),NaN,strip,azithromycin,azithromycin nan,500mg,500mg
2,Ascoril LS Syrup,118.00,Glenmark Pharmaceuticals Ltd,bottle of 100 ml Syrup,Ambroxol (30mg/5ml),Levosalbutamol (1mg/5ml),bottle,ambroxol,ambroxol levosalbutamol,30mg/5ml,30mg/5ml 1mg/5ml
3,Allegra 120mg Tablet,218.81,Sanofi India Ltd,strip of 10 tablets,Fexofenadine (120mg),NaN,strip,fexofenadine,fexofenadine nan,120mg,120mg
4,Avil 25 Tablet,10.96,Sanofi India Ltd,strip of 15 tablets,Pheniramine (25mg),NaN,strip,pheniramine,pheniramine nan,25mg,25mg


In [77]:
# Drop redundant columns
dataset = df.drop(columns=['pack_size_label','short_composition1','short_composition2'])
dataset.head(10)

,name,price(₹),manufacturer_name,type,primary_comp,entire_comp,value,entire_value
0,Augmentin 625 Duo Tablet,223.42,Glaxo SmithKline Pharmaceuticals Ltd,strip,amoxycillin,amoxycillin clavulanic acid,500mg,500mg 125mg
1,Azithral 500 Tablet,132.36,Alembic Pharmaceuticals Ltd,strip,azithromycin,azithromycin nan,500mg,500mg
2,Ascoril LS Syrup,118.00,Glenmark Pharmaceuticals Ltd,bottle,ambroxol,ambroxol levosalbutamol,30mg/5ml,30mg/5ml 1mg/5ml
3,Allegra 120mg Tablet,218.81,Sanofi India Ltd,strip,fexofenadine,fexofenadine nan,120mg,120mg
4,Avil 25 Tablet,10.96,Sanofi India Ltd,strip,pheniramine,pheniramine nan,25mg,25mg
5,Allegra-M Tablet,241.48,Sanofi India Ltd,strip,montelukast,montelukast fexofenadine,10mg,10mg 120mg
6,Amoxyclav 625 Tablet,223.27,Abbott,strip,amoxycillin,amoxycillin clavulanic acid,500mg,500mg 125mg
7,Azee 500 Tablet,132.38,Cipla Ltd,strip,azithromycin,azithromycin nan,500mg,500mg
8,Atarax 25mg Tablet,85.50,Dr Reddy's Laboratories Ltd,strip,hydroxyzine,hydroxyzine nan,25mg,25mg
9,Ascoril D Plus Syrup Sugar Free,129.00,Glenmark Pharmaceuticals Ltd,bottle,phenylephrine,phenylephrine chlorpheniramine maleate,5mg,5mg 2mg


In [78]:
dataset.name.value_counts()

NS 0.9% Infusion             12
Ringer Lactate Infusion       7
Azilife 500mg Tablet          5
Lulizol Cream                 5
Lulibet Cream                 5
                             ..
Flavix 75mg Tablet            1
Flagorin 5000IU Injection     1
Fusiate Cream                 1
Flucos ITZ 100 Capsule        1
Zyvocol 1% Dusting Powder     1
Name: name, Length: 241990, dtype: int64

In [79]:
df.head()

,name,price(₹),manufacturer_name,pack_size_label,short_composition1,short_composition2,type,primary_comp,entire_comp,value,entire_value
0,Augmentin 625 Duo Tablet,223.42,Glaxo SmithKline Pharmaceuticals Ltd,strip of 10 tablets,Amoxycillin (500mg),Clavulanic Acid (125mg),strip,amoxycillin,amoxycillin clavulanic acid,500mg,500mg 125mg
1,Azithral 500 Tablet,132.36,Alembic Pharmaceuticals Ltd,strip of 5 tablets,Azithromycin (500mg),NaN,strip,azithromycin,azithromycin nan,500mg,500mg
2,Ascoril LS Syrup,118.00,Glenmark Pharmaceuticals Ltd,bottle of 100 ml Syrup,Ambroxol (30mg/5ml),Levosalbutamol (1mg/5ml),bottle,ambroxol,ambroxol levosalbutamol,30mg/5ml,30mg/5ml 1mg/5ml
3,Allegra 120mg Tablet,218.81,Sanofi India Ltd,strip of 10 tablets,Fexofenadine (120mg),NaN,strip,fexofenadine,fexofenadine nan,120mg,120mg
4,Avil 25 Tablet,10.96,Sanofi India Ltd,strip of 15 tablets,Pheniramine (25mg),NaN,strip,pheniramine,pheniramine nan,25mg,25mg


In [80]:
dataset['Disp'] = df.apply(lambda row: row['short_composition1'] if pd.isnull(row['short_composition2']) else row['short_composition1'] + ' / ' + row['short_composition2'], axis=1)

In [81]:
dataset[dataset["name"]=='M Lukas FX 10mg/120mg Tablet']

,name,price(₹),manufacturer_name,type,primary_comp,entire_comp,value,entire_value,Disp
139833,M Lukas FX 10mg/120mg Tablet,130.0,Ambience Pharma,strip,montelukast,montelukast fexofenadine,10mg,10mg 120mg,Montelukast (10mg) / Fexofenadine (120mg)


In [85]:
dataset.rename(columns={'name':'Name','Disp':'Composition','manufacturer_name':'Manufacturer Name'}, inplace=True)
dataset.head()

,Name,price(₹),Manufacturer Name,type,primary_comp,entire_comp,value,entire_value,Composition
0,Augmentin 625 Duo Tablet,223.42,Glaxo SmithKline Pharmaceuticals Ltd,strip,amoxycillin,amoxycillin clavulanic acid,500mg,500mg 125mg,Amoxycillin (500mg) / Clavulanic Acid (125mg)
1,Azithral 500 Tablet,132.36,Alembic Pharmaceuticals Ltd,strip,azithromycin,azithromycin nan,500mg,500mg,Azithromycin (500mg)
2,Ascoril LS Syrup,118.00,Glenmark Pharmaceuticals Ltd,bottle,ambroxol,ambroxol levosalbutamol,30mg/5ml,30mg/5ml 1mg/5ml,Ambroxol (30mg/5ml) / Levosalbutamol (1mg/5ml)
3,Allegra 120mg Tablet,218.81,Sanofi India Ltd,strip,fexofenadine,fexofenadine nan,120mg,120mg,Fexofenadine (120mg)
4,Avil 25 Tablet,10.96,Sanofi India Ltd,strip,pheniramine,pheniramine nan,25mg,25mg,Pheniramine (25mg)


In [86]:
dataset[['Name','Composition','Manufacturer Name']]

,Name,Composition,Manufacturer Name
0,Augmentin 625 Duo Tablet,Amoxycillin (500mg) / Clavulanic Acid (125mg),Glaxo SmithKline Pharmaceuticals Ltd
1,Azithral 500 Tablet,Azithromycin (500mg),Alembic Pharmaceuticals Ltd
2,Ascoril LS Syrup,Ambroxol (30mg/5ml) / Levosalbutamol (1mg/5ml),Glenmark Pharmaceuticals Ltd
3,Allegra 120mg Tablet,Fexofenadine (120mg),Sanofi India Ltd
4,Avil 25 Tablet,Pheniramine (25mg),Sanofi India Ltd
...,...,...,...
253968,Ziyapod 100mg Oral Suspension,Cefpodoxime Proxetil (100mg),Ziyana Lifesciences Pvt Ltd
253969,Zemhart 30mg Tablet,Diltiazem (30mg),Leeford Healthcare Ltd
253970,Zivex 25mg Tablet,Hydroxyzine (25mg),Euro Organics
253971,ZI Fast 500mg Injection,Azithromycin (500mg),Burgeon Health Series Private Limited


DATA VISUALIZATION

This is done in Tableau


WORD EMBEDDINGS

In [87]:
from gensim.models import Word2Vec
from tqdm import tqdm

def generate_word_embeddings(column, dataset):
    unique_values = dataset[column].unique()
    sentences = [[str(value)] for value in unique_values] 
    model = Word2Vec(sentences, min_count=1, vector_size=100)
    return model

# Define the text columns for which word embeddings need to be generated
text_columns = ['manufacturer_name', 'type', 'primary_comp','entire_comp', 'value','entire_value']

# Dictionary to store the models
word_embedding_models = {}

# Generate and store word embeddings for each text column
for column in text_columns:
    model = generate_word_embeddings(column, dataset)
    word_embedding_models[column] = model

# Apply word embeddings and create new columns for embeddings
for column, model in tqdm(word_embedding_models.items(), desc="embeddings"):
    new_column_name = column + '_embeddings'
    dataset[new_column_name] = dataset[column].apply(lambda x: model.wv[str(x)].reshape(1, -1) if str(x) in model.wv else [])


KeyError: 'manufacturer_name'

In [88]:
dataset.head()

,Name,price(₹),Manufacturer Name,type,primary_comp,entire_comp,value,entire_value,Composition
0,Augmentin 625 Duo Tablet,223.42,Glaxo SmithKline Pharmaceuticals Ltd,strip,amoxycillin,amoxycillin clavulanic acid,500mg,500mg 125mg,Amoxycillin (500mg) / Clavulanic Acid (125mg)
1,Azithral 500 Tablet,132.36,Alembic Pharmaceuticals Ltd,strip,azithromycin,azithromycin nan,500mg,500mg,Azithromycin (500mg)
2,Ascoril LS Syrup,118.00,Glenmark Pharmaceuticals Ltd,bottle,ambroxol,ambroxol levosalbutamol,30mg/5ml,30mg/5ml 1mg/5ml,Ambroxol (30mg/5ml) / Levosalbutamol (1mg/5ml)
3,Allegra 120mg Tablet,218.81,Sanofi India Ltd,strip,fexofenadine,fexofenadine nan,120mg,120mg,Fexofenadine (120mg)
4,Avil 25 Tablet,10.96,Sanofi India Ltd,strip,pheniramine,pheniramine nan,25mg,25mg,Pheniramine (25mg)


WEIGHTED FEATURES

In [89]:
# These weights can be adjusted based on preference
weightage = {
    'price(₹)': 0.5,
    'manufacturer_name_embeddings': 0.3,
    'type_embeddings': 1.0,
    'primary_comp_embeddings': 0.8,
    'entire_comp_embeddings':1.0,
    'value_embeddings': 0.8,
    'entire_value_embeddings': 1.0
}

GENERATING RECOMMENDATIONS


In [90]:
# Calculate similarity scores for 'price' column using cosine similarity
def calculate_price_similarity(price1, price2):
    price1 = np.array(price1).reshape(1, -1)
    price2 = np.array(price2).reshape(1, -1)
    similarity = cosine_similarity(price1, price2)
    return similarity[0][0]

def calculate_text_similarity(value1, value2):
    similarity = cosine_similarity(value1, value2)
    return similarity[0][0]

# Calculate weighted similarity between two medicines
def calculate_weighted_similarity(medicine1, medicine2):
    similarity_scores = []
    for column, weight in weightage.items():
        if column == 'price(₹)':
            similarity = calculate_price_similarity(medicine1[column].values[0], medicine2[column])
        else:
            similarity = calculate_text_similarity(medicine1[column].values[0], medicine2[column])
        similarity_scores.append(similarity * weight)
    weighted_similarity = sum(similarity_scores)
    return weighted_similarity

In [91]:
search_for_recommendations = 'M Lukas FX 10mg/120mg Tablet'

In [92]:
# Running this cell will take some time (~5-10 mins)
medicine = dataset[dataset['Name'] == search_for_recommendations]

medicine['Disp'].values[0]

KeyError: 'name'

In [63]:
if medicine['primary_comp'].values[0] == medicine['entire_comp'].values[0].rstrip():
    del weightage['primary_comp_embeddings']

if medicine['value'].values[0] == medicine['entire_value'].values[0].rstrip():
    del weightage['value_embeddings']
new_dataset = dataset.copy(deep=True)
new_dataset['similarity_score'] = dataset.apply(lambda x: calculate_weighted_similarity(medicine, x), axis=1)
top_10_similar_medicines = new_dataset.nlargest(11, 'similarity_score')[1:]

In [64]:
print(f"Top 10 Similar Medicines to '{search_for_recommendations}':")
print(top_10_similar_medicines[['name', 'similarity_score']])

Top 10 Similar Medicines to 'M Lukas FX 10mg/120mg Tablet':
                                  name  similarity_score
87065      Fexofit M 10mg/120mg Tablet          5.201111
139627   Montonice F 10mg/120mg Tablet          5.190806
30936       Bmont FX 10mg/120mg Tablet          5.183748
89813     Fexowish M 10mg/120mg Tablet          5.177685
140316        Mac FX 10mg/120mg Tablet          5.176201
137629  Montilife FX 10mg/120mg Tablet          5.175996
143995      Mojer FX 10mg/120mg Tablet          5.171141
146677     Monzer FX 10mg/120mg Tablet          5.171141
5                     Allegra-M Tablet          5.171009
88586                  Fekast-M Tablet          5.169733


In [65]:
top_10_similar_medicines.head(10)

,name,price(₹),manufacturer_name,type,primary_comp,entire_comp,value,entire_value,Disp,manufacturer_name_embeddings,type_embeddings,primary_comp_embeddings,entire_comp_embeddings,value_embeddings,entire_value_embeddings,similarity_score
87065,Fexofit M 10mg/120mg Tablet,149.90,Blaine Pharmaceuticals Pvt. Ltd,strip,montelukast,montelukast fexofenadine,10mg,10mg 120mg,Montelukast (10mg) / Fexofenadine (120mg),"[[-0.0076312064, 0.0071488726, 0.008740047, 0....","[[-0.0071909428, 0.0042328904, 0.0021633946, 0...","[[0.008164572, 0.0019218302, -0.0073633813, -0...","[[0.006607245, 0.002543224, -0.0007322073, -0....","[[-0.00302428, -0.006213844, 0.002317704, -0.0...","[[-0.008116465, 0.0096094515, 0.0050239847, 0....",5.201111
139627,Montonice F 10mg/120mg Tablet,99.50,Fidalgo Laboratories Pvt Ltd,strip,montelukast,montelukast fexofenadine,10mg,10mg 120mg,Montelukast (10mg) / Fexofenadine (120mg),"[[-0.0034892678, -0.0009659004, -0.0038359975,...","[[-0.0071909428, 0.0042328904, 0.0021633946, 0...","[[0.008164572, 0.0019218302, -0.0073633813, -0...","[[0.006607245, 0.002543224, -0.0007322073, -0....","[[-0.00302428, -0.006213844, 0.002317704, -0.0...","[[-0.008116465, 0.0096094515, 0.0050239847, 0....",5.190806
30936,Bmont FX 10mg/120mg Tablet,120.00,Ambrosia Pharmaceuticals,strip,montelukast,montelukast fexofenadine,10mg,10mg 120mg,Montelukast (10mg) / Fexofenadine (120mg),"[[0.0038967251, 0.0068735685, 0.002461226, -0....","[[-0.0071909428, 0.0042328904, 0.0021633946, 0...","[[0.008164572, 0.0019218302, -0.0073633813, -0...","[[0.006607245, 0.002543224, -0.0007322073, -0....","[[-0.00302428, -0.006213844, 0.002317704, -0.0...","[[-0.008116465, 0.0096094515, 0.0050239847, 0....",5.183748
89813,Fexowish M 10mg/120mg Tablet,149.00,Nusmith Pharma Pvt. Ltd,strip,montelukast,montelukast fexofenadine,10mg,10mg 120mg,Montelukast (10mg) / Fexofenadine (120mg),"[[-0.0025384652, -0.0005206823, 0.009051293, 0...","[[-0.0071909428, 0.0042328904, 0.0021633946, 0...","[[0.008164572, 0.0019218302, -0.0073633813, -0...","[[0.006607245, 0.002543224, -0.0007322073, -0....","[[-0.00302428, -0.006213844, 0.002317704, -0.0...","[[-0.008116465, 0.0096094515, 0.0050239847, 0....",5.177685
140316,Mac FX 10mg/120mg Tablet,138.00,Zicad Life Care,strip,montelukast,montelukast fexofenadine,10mg,10mg 120mg,Montelukast (10mg) / Fexofenadine (120mg),"[[-0.005214183, 0.006548977, 0.006083438, 0.00...","[[-0.0071909428, 0.0042328904, 0.0021633946, 0...","[[0.008164572, 0.0019218302, -0.0073633813, -0...","[[0.006607245, 0.002543224, -0.0007322073, -0....","[[-0.00302428, -0.006213844, 0.002317704, -0.0...","[[-0.008116465, 0.0096094515, 0.0050239847, 0....",5.176201
137629,Montilife FX 10mg/120mg Tablet,144.65,MDC Pharmaceuticals Pvt Ltd,strip,montelukast,montelukast fexofenadine,10mg,10mg 120mg,Montelukast (10mg) / Fexofenadine (120mg),"[[-0.005395074, 0.0053055226, 0.007699547, 0.0...","[[-0.0071909428, 0.0042328904, 0.0021633946, 0...","[[0.008164572, 0.0019218302, -0.0073633813, -0...","[[0.006607245, 0.002543224, -0.0007322073, -0....","[[-0.00302428, -0.006213844, 0.002317704, -0.0...","[[-0.008116465, 0.0096094515, 0.0050239847, 0....",5.175996
143995,Mojer FX 10mg/120mg Tablet,130.00,JV Healthcare,strip,montelukast,montelukast fexofenadine,10mg,10mg 120mg,Montelukast (10mg) / Fexofenadine (120mg),"[[-0.008031942, -0.0016422117, -0.004198711, 0...","[[-0.0071909428, 0.0042328904, 0.0021633946, 0...","[[0.008164572, 0.0019218302, -0.0073633813, -0...","[[0.006607245, 0.002543224, -0.0007322073, -0....","[[-0.00302428, -0.006213844, 0.002317704, -0.0...","[[-0.008116465, 0.0096094515, 0.0050239847, 0....",5.171141
146677,Monzer FX 10mg/120mg Tablet,130.00,JV Healthcare,strip,montelukast,montelukast fexofenadine,10mg,10mg 120mg,Montelukast (10mg) / Fexofenadine (120mg),"[[-0.008031942, -0.0016422117, -0.004198711, 0...","[[-0.0071909428, 0.0042328904, 0.0021633946, 0...","[[0.008164572, 0.0019218302, -0.0073633813, -0...","[[0.006607245, 0.002543224, -0.00073